In [1]:
from __future__ import division
%matplotlib nbagg
import os, sys
import numpy as np
import pandas as pd
from collections import defaultdict
from python_latexify import latexify, format_axes
import matplotlib
matplotlib.rcParams.update(latexify())
import math
from censorship_ratios import *

#font = {'family' : 'normal',
#        'weight' : 'bold',
#        'size'   : 14}
#matplotlib.rc('font', **font)
import matplotlib.pyplot as plt

In [2]:
DATA = "data/"
PLOTS = "results/final_figures-20150429/plots-580sIPs/"
TABLES = "results/final_figures-20150429/tables-580sIPs/"

if not os.path.exists(PLOTS):
    os.makedirs(PLOTS)
if not os.path.exists(TABLES):
    os.makedirs(TABLES)

In [75]:
# UPDATED SERVERS IMC
df_sIP_subcat = pd.read_csv("data/Servers_IMC.txt.original")
df_sIP_subcat['subcat'] = df_sIP_subcat['subcat'].apply(lambda x: x.replace('kids_and_teens', 'teens'))


subcat_duplicated = defaultdict(list)
for ix, row in df_sIP_subcat.iterrows():
    if len(row[2].split("|"))>1:
        shared = 1
    else:
        shared = 0
    for subcat in row[2].split("|"):
        if subcat == 'kids_and_teens':
            subcat = 'teens'
        subcat_duplicated['sIP'].append(row[0])
        subcat_duplicated['domain'].append(row[1])
        subcat_duplicated['subcat'].append(subcat)
        subcat_duplicated['shared'].append(shared)
df_sIP_subcat_disjoint = pd.DataFrame(subcat_duplicated)

In [76]:
print len(df_sIP_subcat), len(df_sIP_subcat_disjoint)
valid_sIPs = df_sIP_subcat['sIP'].unique()
print "VALID sIPs Unique = ", len(valid_sIPs), len(df_sIP_subcat_disjoint['sIP'].unique())

580 875
VALID sIPs Unique =  580 580


In [77]:
df_all1 = pd.read_pickle(DATA + "case_detected_all_20150423.pkl")
df_all2 = pd.read_pickle(DATA + "case_detected_all_20150427.pkl")
df_all = pd.concat([df_all1,df_all2])
print len(df_all1), len(df_all2), len(df_all)

250593 146328 396921


In [78]:
COLS_NEEDED = ['sIP', 'domain', 'subcat', 'country', 'case', 'port']

df_sanitized = df_all[df_all['sIP'].isin(valid_sIPs)].reset_index()[COLS_NEEDED]
df_disjoint = df_sanitized[['sIP', 'country', 'case', 'port']].merge(
    df_sIP_subcat_disjoint[['sIP', 'domain', 'subcat']], on=['sIP']).reset_index()

In [79]:
print len(df_all), len(df_sanitized), len(df_disjoint)

396921 396921 605854


In [80]:
df_censorship, df_technology, df_unknown = get_censorship_technology_sIP(df_all)
df_tot = get_censorship_by_country_sIP(df_all, 'tot')

In [122]:
temp = df_censorship.apply(np.nanmedian, axis=1)
bad_sIP2 = temp[temp > 0]
len(bad_sIP2)

95

In [126]:
len(bad_sIP2.ix[bad_sIP])

95

In [98]:
bad_sIP = df_censorship[ df_censorship.median(axis=1) > 0 ].index
len(df_sIP_subcat[df_sIP_subcat['sIP'].isin(bad_sIP)])

95

In [99]:
good_sIP = df_censorship[ df_censorship.median(axis=1) == 0 ].index
len(df_sIP_subcat[df_sIP_subcat['sIP'].isin(good_sIP)])

424

In [117]:
np.median([np.nan, np.nan, np.nan, 0, 1])

nan

In [114]:
df_bad = df_all[df_all['sIP'].isin(bad_sIP)][['gIP', 'sIP', 'country', 'domain', 'subcat','case',\
'port']].groupby(['sIP', 'domain', 'subcat', 'case'])['port'].sum().unstack()
df_bad['tot'] = df_bad.sum(axis=1)
df_bad

,,case,0,1,2,3,4,tot
sIP,domain,subcat,,,,,,
104.16.5.13,change.org,society,800,480,160,37120,1760,40320
104.16.7.118,petmd.com,health,1040,240,80,36560,1520,39440
104.20.12.28,studentdoctor.net,health,800,400,160,36880,1760,40000
104.20.28.30,bungie.net,kids_and_teens|games,720,400,NaN,36080,2320,39520
104.20.28.34,tunnelbear.com,vpn,720,400,80,37520,960,39680
104.20.5.196,furaffinity.net,adult,880,480,NaN,36960,2160,40480
104.28.7.85,indiansexstories.net,adult,1280,240,NaN,37280,1520,40320
107.20.154.40,strava.com,sports,320,14720,NaN,80,480,15600
107.20.157.190,suicidegirls.com,adult,240,6560,18000,400,400,25600


# CENSORSHIP RATIO COMPONENTS BASED ON APR23 APR27

In [55]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
mat2 = pca.fit_transform(mat)

In [57]:
pca.explained_variance_

array([ 4.24592937,  1.41734802,  0.86396505,  0.62526924,  0.57625987,
        0.51425905,  0.50051922,  0.4659367 ,  0.4272712 ,  0.40937925])

In [23]:
import scipy.stats as stats
from numpy.linalg import svd
import collections
import pprint

# load the domain to category lookups
dom2cat, cat2dom = {}, {}
with open('utils/domain-to-cat-2015-march-27.txt', 'r') as filep:
    for line in filep.readlines():
        domain, cats = line.strip().split(',')
        cats = cats.split('|')
        dom2cat[domain] = cats
        for cat in cats:
            if cat not in cat2dom:
                cat2dom[cat] = {}
            cat2dom[cat][domain] = True
            
# load the country to region tables
cnt2reg, reg2cnt, reg2name = {}, {}, {}
with open('utils/imf-region-to-country.txt', 'r') as filep:
    for line in filep.readlines():
        reg, cc = line.strip().split(',')
        cnt2reg[cc] = reg
        if reg not in reg2cnt:
            reg2cnt[reg] = {}
        reg2cnt[reg][cc] = True
with open('utils/imf-region-names.txt', 'r') as filep:
    for line in filep.readlines():
        reg, abrev = line.strip().split(',')
        reg2name[abrev] = reg

def center(a, scale=True, axis=0):
    """Center the matrix so that units don't matter by subtracting the mean and
    dividing by the standard deviation
    
    Again, this won't tell us the most commonly censored sites, this will tell us
    which sites are commonly censored together
    
    """
    b = np.array(a, copy=True)
    mean = a.mean(axis=axis)
    b -= mean
    if scale:
        std = b.std(axis=axis)
        std = np.where( std, std, 1. )
        b /= std
    return b

In [50]:
df_censorship, df_technology, df_unknown = get_censorship_technology_sIP(df_sanitized)
df_tot = get_censorship_by_country_sIP(df_sanitized, 'tot')

df = df_technology

In [51]:
# compute the principal components
countries = df.columns
sIPs = df.index
sIP2site = df_sIP_subcat.set_index('sIP')['domain']
sites = [sIP2site[x] for x in sIPs]

mat = df.T.fillna(0).as_matrix() #rows: countries, cols: all sIPs
center_obj = center(mat)
U, s, Vt = svd(center_obj)

V = Vt.T


# print some info about the top few principal components
print "Variance explained, mean, std dev of vector"
for index in range(10):
    var_exp = s[index] * s[index] / float(len(countries))
    mean = stats.tmean(V[:, index])
    var = stats.tvar(V[:, index])
    # compute how many elements are really present in the vector
    stuff = [1 for x in V[:, index] if math.fabs(x) > 0.05]
    
    # print it all out
    print "{:< .5e}, {:< .5e}, {:< .5e}, {}".format(var_exp, mean, var, len(stuff))

Variance explained, mean, std dev of vector
 5.53870e+01, -5.36706e-02,  4.43195e-04, 176
 2.05873e+01, -6.78810e-03,  3.28710e-03, 131
 1.25424e+01, -4.49492e-03,  3.31306e-03, 128
 1.20017e+01, -6.20316e-04,  3.33295e-03, 104
 1.09839e+01,  3.72701e-03,  3.31940e-03, 107
 9.58883e+00,  3.38245e-03,  3.32185e-03, 97
 8.27615e+00,  2.27728e-03,  3.32813e-03, 115
 7.27063e+00, -2.97043e-04,  3.33324e-03, 102
 6.94324e+00, -1.41419e-03,  3.33133e-03, 97
 6.63433e+00,  1.75775e-03,  3.33023e-03, 106


In [52]:
# let's explore the individual components in more depth by printing the top 9 components (don't care about first component)
for index in range(0, 10):
    mean = stats.tmean(V[:, index])
    indices = [x for x in range(len(V[:, index])) if math.fabs(V[x, index]) > 0.05]
    site_list = [sites[x] for x in indices]
    counter = collections.Counter()
    for site in site_list:
        cats = dom2cat[site]
        for cat in cats:
            counter[cat] += 1
    print "Component " + str(index) + ": " +", ".join(site_list) + "\n"
    print sorted(counter.items(), key=lambda x: x[1], reverse=True)
    print
    entries = {}
    for site in site_list:
        cats = dom2cat[site]
        for cat in cats:
            if cat not in entries:
                entries[cat] = []
            entries[cat].append(site)
    #pprint.pprint(entries)

Component 0: express-vpn.com, cupcakebridge.com, alexa.com, mnn.com, pinterest.com, rollingstone.com, www.dropbox.com, thesafety.us, planetsuzy.org, vintage-erotica-forum.com, hindustantimes.com, cornell.edu, columbia.edu, arxiv.org, washington.edu, mayoclinic.org, torDir, xbox.com, bom.gov.au, hm.com, epa.gov, usgs.gov, ieee.org, williamhill.com, att.com, hp.com, uj.ac.za, bettycrocker.com, ups.com, fidelity.com, cnn.com, cartoonnetwork.com, money.cnn.com, nba.com, squirt.org, timeout.com, shutterstock.com, ed.gov, macys.com, space.com, costco.com, staples.com, marriott.com, travelzoo.com, healthcare.gov, fafsa.ed.gov, www.etsy.com, bravenewsoftware.org, cbc.ca, individual.com, virtua-fighter-4.com, huisgenoot.com, zillow.com, alkasir.com, spotify.com, globaleaks.org, stardoll.com, ar.rian.ru, cricket.co.za, kyknet.dstv.com, spine-health.com, sulekha.com, bhg.com, travel.state.gov, stackoverflow.com, infowars.com, bitmask.net, deviantart.com, go.com, games.espn.go.com, thehun.net, tor

In [53]:
import collections
from incf.countryutils import transformations
import pprint
# now try clustering together similar countries

cnt_site = center(mat.T)
Uc, sc, Vtc = svd(cnt_site)

Vc = Vtc.T


# print some info about the top few principal components
print "Variance explained, mean, std dev of vector"
for index in range(10):
    var_exp = sc[index] * sc[index] / float(len(sites))
    mean = stats.tmean(Vc[:, index])
    var = stats.tvar(Vc[:, index])
    # compute how many elements are really present in the vector
    stuff = [1 for x in Vc[:, index] if math.fabs(x) > 0.05]
    
    # print it all out
    print "{:< .5e}, {:< .5e}, {:< .5e}, {}".format(var_exp, mean, var, len(stuff))


Variance explained, mean, std dev of vector
 2.13586e+01,  7.72843e-02,  7.89232e-04, 121
 1.08388e+01,  7.16484e-03,  6.75104e-03, 79
 9.34088e+00,  5.72530e-03,  6.76972e-03, 79
 6.48933e+00, -1.02660e-02,  6.69661e-03, 97
 3.79409e+00,  2.91987e-03,  6.79414e-03, 87
 3.13532e+00, -9.72847e-06,  6.80272e-03, 77
 2.59186e+00, -9.52845e-04,  6.80181e-03, 75
 2.50699e+00,  1.17934e-03,  6.80132e-03, 74
 2.32264e+00,  1.22853e-03,  6.80120e-03, 72
 2.16857e+00,  5.38538e-03,  6.77352e-03, 75


In [54]:
with open('utils/imf-region-to-country.txt', 'r') as filep:
    for line in filep.readlines():
        reg, cc = line.strip().split(',')
        cnt2reg[cc] = reg
        if reg not in reg2cnt:
            reg2cnt[reg] = {}
        reg2cnt[reg][cc] = True
        
cnt2reg['global']='gl'
cnt2reg['cu']='sa'
cnt2reg['so']='af'
cnt2reg['pg']='as'
reg2name['gl']='Global'

# let's explore the individual components in more depth by printing the top 9 components 
for index in range(0, 10):
    mean = stats.tmean(Vc[:, index])
    indices = [x for x in range(len(Vc[:, index])) if math.fabs(Vc[x, index]) > 0.05]
    cnt_list = [countries[x] for x in indices]
#    cnt_names = [transformations.cc_to_cn(x) for x in cnt_list]
    reg_cnts = collections.Counter([reg2name[cnt2reg[x.lower()]] for x in cnt_list if x != "N/A"])

    cnt_names = []
    for x in cnt_list:
        if x == "N/A":
            continue
        cnt_names.append('Global' if x=='global' else transformations.cc_to_cn(x))
    explained_var = sc[index] * sc[index] / float(len(sites))
    print "Component {}: explained var: {} countries: {}".format(index, explained_var, ", ".join(cnt_list))
    print ", ".join(cnt_names) + "\n"
    print sorted(reg_cnts.items(), key=lambda x: x[1], reverse=True)
    print


Component 0: explained var: 21.3586049567 countries: AL, AM, AR, AT, AU, AZ, BA, BE, BG, BJ, BO, BR, CD, CG, CH, CI, CL, CM, CN, CO, CR, CZ, DE, DK, DO, DZ, EC, EE, ES, FI, FR, GA, GB, GE, GH, GQ, GR, HK, HN, HR, HU, ID, IL, IN, IQ, IR, IT, JM, JO, JP, KE, KG, KH, KW, KZ, LA, LB, LK, LR, LS, LT, LV, LY, MA, MD, MG, MK, ML, MM, MN, MR, MU, MX, MY, MZ, NA, NE, NG, NL, NO, NZ, OM, PA, PE, PH, PK, PL, PS, PT, PY, QA, RO, RS, RU, RW, SA, SE, SG, SI, SK, SL, SN, SO, SY, TG, TH, TL, TM, TN, TT, TW, TZ, UA, UY, UZ, VE, VN, ZA, ZM, ZW, global
Albania, Armenia, Argentina, Austria, Australia, Azerbaijan, Bosnia and Herzegovina, Belgium, Bulgaria, Benin, Bolivia, Brazil, Congo, Congo, Switzerland, Cote d'Ivoire, Chile, Cameroon, China, Colombia, Costa Rica, Czech Republic, Germany, Denmark, Dominican Republic, Algeria, Ecuador, Estonia, Spain, Finland, France, Gabon, United Kingdom of Great Britain & Northern Ireland, Georgia, Ghana, Equatorial Guinea, Greece, Hong Kong, Honduras, Croatia, Hungary

# LOOKING AT CIRCUM

In [10]:
# list of sIP by subcat: use disjoint
def select_sIP_in_subcat(df_sIP_subcat, subcat='circum_vpn'):
    # select both circum or vpn
    sIP_by_subcat = df_sIP_subcat.groupby('subcat')['sIP'].unique()
    #subcat = 'circum|vpn'
    sliced_sIP = []
    for categ in subcat.split('_'):
        sliced_sIP += list(sIP_by_subcat[categ])
    print subcat, sliced_sIP
    return sliced_sIP

In [11]:
sliced_sIP = select_sIP_in_subcat(df_sIP_subcat_disjoint, subcat='circum')

circum ['216.239.38.21', '81.169.218.3', '205.204.94.195', '50.62.247.1', '204.13.164.188', '104.193.9.122', '1.209.122.23', '94.23.47.218', '38.229.72.16', '107.170.93.98', '82.196.1.165', '205.251.253.116', '199.119.112.23', '192.249.116.218', '54.230.219.62', '173.236.226.119', '209.234.253.125', '216.239.32.21', '194.150.168.119', '82.195.75.101', '80.68.94.117', '192.69.95.83', '38.101.236.62', '86.59.21.38', '194.109.206.212', '131.188.40.189', '193.23.244.244', '154.35.175.225', '199.254.238.52']


In [18]:
df_censorship, df_technology, df_unknown = get_censorship_technology_sIP(df_sanitized)
df_tot = get_censorship_by_country_sIP(df_sanitized, 'tot')

In [21]:
df_tot.ix[sliced_sIP]

country,AE,AF,AL,AM,AO,AR,AT,AU,AZ,BA,...,UG,US,UY,UZ,VE,VN,ZA,ZM,ZW,global
sIP,,,,,,,,,,,,,,,,,,,,,
216.239.38.21,4,7,6,7,7,7,8,5,7,7,...,7,5,7,7,7,7,7,7,7,891
81.169.218.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205.204.94.195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50.62.247.1,6,7,6,7,6,7,7,3,4,7,...,6,3,2,7,6,6,4,6,6,858
204.13.164.188,3,7,2,5,7,7,3,6,4,7,...,6,1,4,7,6,7,4,6,2,760
104.193.9.122,1,7,4,2,7,4,6,4,5,7,...,7,4,4,6,6,3,2,4,6,742
1.209.122.23,1,7,6,7,6,6,7,8,3,7,...,7,8,7,6,6,6,7,5,7,886
94.23.47.218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38.229.72.16,2,6,3,5,6,4,4,3,4,6,...,6,3,6,5,4,4,3,5,5,733


In [23]:
censor = df_censorship.ix[sliced_sIP]
censor[censor>0.9]

country,AE,AF,AL,AM,AO,AR,AT,AU,AZ,BA,...,UG,US,UY,UZ,VE,VN,ZA,ZM,ZW,global
sIP,,,,,,,,,,,,,,,,,,,,,
216.239.38.21,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,...,1,NaN,NaN,1,NaN,1,NaN,1,NaN,NaN
81.169.218.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205.204.94.195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50.62.247.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204.13.164.188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104.193.9.122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.209.122.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94.23.47.218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38.229.72.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
censor = (get_censorship_by_country_sIP(df_sanitized, 1)+get_censorship_by_country_sIP(df_sanitized, 3))/get_censorship_by_country_sIP(df_sanitized, 'tot')

In [42]:
censor[censor>0.9].stack()

sIP            country
216.239.38.21  AE         1
               AF         1
               AO         1
               BA         1
               BD         1
               BF         1
               BH         1
               BI         1
               BW         1
               BY         1
               CA         1
               CD         1
               CI         1
               CL         1
               CM         1
...
216.239.32.21    ZM         1
194.150.168.119  LR         1
38.101.236.62    CN         1
194.109.206.212  IR         1
                 SA         1
131.188.40.189   BE         1
                 CN         1
                 LR         1
193.23.244.244   CN         1
                 IR         1
                 LR         1
                 MD         1
                 TM         1
199.254.238.52   CN         1
                 LR         1
Length: 188, dtype: float64